# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data

In [1]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data


In [4]:
view_sentence_range = (20, 30)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 20 to 30:


Moe_Szyslak: Looks like this is the end.
Barney_Gumble: That's all right. I couldn't have led a richer life.
Barney_Gumble: So the next time somebody tells you county folk are good, honest people, you can spit in their faces for me!
Lisa_Simpson: I will, Mr. Gumbel. But if you'll excuse me, I'm profiling my dad for the school paper. I thought it would be neat to follow him around for a day to see what makes him tick.
Barney_Gumble: Oh, that's sweet. I used to follow my dad to a lot of bars too. (BELCH)
Moe_Szyslak: Here you go. One beer, one chocolate milk.
Lisa_Simpson: Uh, excuse me, I have the chocolate milk.
Moe_Szyslak: Oh.


## Preprocessing


### Lookup Table

- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [5]:
import numpy as np
import common
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """    
    vocab_to_int, int_to_vocab = common.create_lookup_tables(text)
    
    return vocab_to_int, int_to_vocab


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """    
    x = {
        '.'  : '||period||',
        ','  : '||comma||',
        '"'  : '||quote||',
        '('  : '||left_bracket||',
        '?'  : '||question_mark||',
        '!'  : '||exclamation||',
        '\n' : '||new_line||',
        ')'  : '||right_bracket||',
        ';'  : '||semi-colon||' ,
        '--' : '||dash_dash||'
        }
    
    return x

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [7]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [8]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
### Check the Version of TensorFlow and Access to GPU

In [9]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input

In [42]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input_ = tf.placeholder(tf.int32, [None, None], name='input')
    
    targets = tf.placeholder(tf.int32, [None, None], name ='targets')
    
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return input_, targets, learning_rate


tests.test_get_inputs(get_inputs)

Tests Passed


### Word Embedding

In [25]:
def get_embed(input_, n_vocab, n_embedding):
    """
    Create embedding for <input_data>.
    :param input_: TF placeholder for text input.
    :param n_vocab: Number of words in vocabulary.
    :param n_embedding: Number of embedding dimensions
    :return: Embedded input.
    """
    
    embedding = tf.Variable( tf.random_uniform( (n_vocab, n_embedding)))
    embed = tf.nn.embedding_lookup( embedding, input_) 
    
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN Cell and Initialize

In [12]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """

    lstms = [tf.contrib.rnn.BasicLSTMCell(rnn_size)]
    cell = tf.contrib.rnn.MultiRNNCell( lstms )

    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    
    return cell, initial_state


tests.test_get_init_cell(get_init_cell)

Tests Passed


### Build RNN

In [35]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
   
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)   
    state = tf.identity(state, name="final_state")
    
    return outputs, state

tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [37]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    
    embed = get_embed(input_data, vocab_size, embed_dim)
    
    outputs, state = build_rnn(cell, embed)
    
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None,
                                              weights_initializer = tf.truncated_normal_initializer(
                                                  mean = 0.0, stddev = .1),
                                              biases_initializer=tf.zeros_initializer() )
    
    return logits, state


tests.test_build_nn(build_nn)

Tests Passed


## Batches

In [77]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    
    int_text = int_text[0:(len(int_text) - len(int_text) % (batch_size * seq_length))]
    
    targets = np.zeros(len(int_text)).astype(int)
    targets[:-1] = int_text[1:]
    targets[-1] = int_text[0]
    
    elements_per_batch = batch_size * seq_length
    
    num_batches = int(len(int_text) / elements_per_batch)
    
    # Build the batched data
    batches = np.zeros((num_batches, 2, batch_size, seq_length)).astype(int)
    
    for sequence in range(batch_size):

        for batch in range(num_batches):
            
            start_at = (batch * seq_length) + ( sequence * seq_length * num_batches )
            # Append to inputs
            batches[batch, 0, sequence, :] = int_text[start_at:(start_at + seq_length)]
            # Append to targets
            batches[batch, 1, sequence, :] = targets[start_at:(start_at + seq_length)]
    
    return batches
    

tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters

In [101]:
num_epochs = 300
batch_size = 128
rnn_size = 256
embed_dim = 200
seq_length = 15
learning_rate = .001
show_every_n_batches = 100

save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [102]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [103]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/35   train_loss = 8.894
Epoch   2 Batch   30/35   train_loss = 5.554
Epoch   5 Batch   25/35   train_loss = 4.952
Epoch   8 Batch   20/35   train_loss = 4.822
Epoch  11 Batch   15/35   train_loss = 4.565
Epoch  14 Batch   10/35   train_loss = 4.382
Epoch  17 Batch    5/35   train_loss = 4.121
Epoch  20 Batch    0/35   train_loss = 4.061
Epoch  22 Batch   30/35   train_loss = 3.896
Epoch  25 Batch   25/35   train_loss = 3.681
Epoch  28 Batch   20/35   train_loss = 3.607
Epoch  31 Batch   15/35   train_loss = 3.515
Epoch  34 Batch   10/35   train_loss = 3.416
Epoch  37 Batch    5/35   train_loss = 3.221
Epoch  40 Batch    0/35   train_loss = 3.159
Epoch  42 Batch   30/35   train_loss = 3.045
Epoch  45 Batch   25/35   train_loss = 2.916
Epoch  48 Batch   20/35   train_loss = 2.811
Epoch  51 Batch   15/35   train_loss = 2.734
Epoch  54 Batch   10/35   train_loss = 2.669
Epoch  57 Batch    5/35   train_loss = 2.575
Epoch  60 Batch    0/35   train_loss = 2.487
Epoch  62 

## Save Parameters

In [104]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [105]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [106]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    
    tensors = ["input:0", "initial_state:0", "final_state:0", "probs:0"]
    output = []
    
    for t in tensors:
        output.append( loaded_graph.get_tensor_by_name(t) )

    return output[0], output[1], output[2], output[3]


tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [107]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    return int_to_vocab[np.argmax(probabilities)]

tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [108]:
gen_length = 300
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
homer_simpson:(very up) hi guys!
homer_simpson: moe, get the darts.
c. _montgomery_burns: boy, i was the luckiest guy i know.
moe_szyslak: i'm spiffin' up the bad guy.
homer_simpson: and i will say that.
kemi: also.
marge_simpson: hmmm.
moe_szyslak: what are you talking about?
duffman: now, i'm off your guy, dr.
homer_simpson: i could do, huh?... what?(looks around) how we are very good for granted.
marge_simpson: no... that's all right... a little different?
lenny_leonard: you mean like how we treat your" friend?" and finally" the springfield community playhouse was perfect.
kent_brockman:(to moe) oh, shut up.
tv_husband:(from tv) no, you shut up.
kent_brockman: so when you realized we hadn't had no ladies in my brain.
moe_szyslak: no, no, it ain't no mystery-- whole modern world got a swishifyin' meal, the next day from an island off the coast of massachusetts...


moe_szyslak: if you all have to know that?
homer_simpson:(slyly) i thin

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.